# 1. Despligue de un cluster de Vault en minikube (sobre Docker)

### Basado en el [tutorial](https://developer.hashicorp.com/vault/tutorials/kubernetes/kubernetes-minikube-tls)

Eliminamos el perfil webinar

In [ ]:
! minikube delete -p webinar

Iniciamos Podman desktop

In [ ]:
! open -a "Podman Desktop"

In [ ]:
%%bash
podman machine init --cpus 4 --memory 4096 --disk-size 20
podman machine start
podman system connection default podman-machine-default-root
podman info

In [ ]:
! minikube start -p webinar

Por defecto kubectl debería estar apuntando a este clúster, pero en cualquier caso lo vamos a definir explícitamente

In [ ]:
! kubectl config use-context webinar

Añadimos el repo de HashiCorp para a continuación poder instalar Vault vía helm

In [ ]:
%%bash
helm repo add hashicorp https://helm.releases.hashicorp.com
helm repo update

Definimos una serie de variable de entorno con las que trabajaremos a continuación

In [ ]:
%env WORKDIR=/tmp/vault
%env VAULT_K8S_NAMESPACE=vault
%env VAULT_HELM_RELEASE_NAME=vault
%env VAULT_SERVICE_NAME=vault-internal 
%env K8S_CLUSTER_NAME=cluster.local 

Creamos un directorio temporal donde se almacenaran los certificados, claves de Shamir y root token de Vault

In [ ]:
%%bash
rm -rf /tmp/vault
mkdir /tmp/vault

El siguiente paso será la generación de certificados para el clúster de Vault. Usaremos la CA de Kubernetes para firmar dichos certificados. Los ficheros resultantes se guardar en el directorio creado en el paso previo. Como primer paso, procedermos con la creación de un namespace dedicado a Vault.

In [ ]:
! kubectl create namespace $VAULT_K8S_NAMESPACE

In [ ]:
%%bash

openssl genrsa -out ${WORKDIR}/vault.key 2048
cat > ${WORKDIR}/vault-csr.conf <<EOF
[req]
default_bits = 2048
prompt = no
encrypt_key = yes
default_md = sha256
distinguished_name = kubelet_serving
req_extensions = v3_req
[ kubelet_serving ]
O = system:nodes
CN = system:node:*.${VAULT_HELM_RELEASE_NAME}.svc.${K8S_CLUSTER_NAME}
[ v3_req ]
basicConstraints = CA:FALSE
keyUsage = nonRepudiation, digitalSignature, keyEncipherment, dataEncipherment
extendedKeyUsage = serverAuth, clientAuth
subjectAltName = @alt_names
[alt_names]
DNS.1 = *.${VAULT_SERVICE_NAME}
DNS.2 = *.${VAULT_SERVICE_NAME}.${VAULT_HELM_RELEASE_NAME}.svc.${K8S_CLUSTER_NAME}
DNS.3 = *.${VAULT_HELM_RELEASE_NAME}
DNS.4 = *.${VAULT_HELM_RELEASE_NAME}.svc.${K8S_CLUSTER_NAME}
IP.1 = 127.0.0.1
EOF

openssl req -new -key ${WORKDIR}/vault.key -out ${WORKDIR}/vault.csr -config ${WORKDIR}/vault-csr.conf


cat > ${WORKDIR}/csr.yaml <<EOF
apiVersion: certificates.k8s.io/v1
kind: CertificateSigningRequest
metadata:
   name: vault.svc
spec:
   signerName: kubernetes.io/kubelet-serving
   # signerName: kubernetes.io/kubelet-serving
   expirationSeconds: 8640000
   request: $(cat ${WORKDIR}/vault.csr|base64|tr -d '\n')
   usages:
   - digital signature
   - key encipherment
   - server auth
EOF

kubectl create -f ${WORKDIR}/csr.yaml
kubectl certificate approve vault.svc
kubectl get csr vault.svc
kubectl get csr vault.svc -o jsonpath='{.status.certificate}' | openssl base64 -d -A -out ${WORKDIR}/vault.crt
kubectl config view \
--raw \
--minify \
--flatten \
-o jsonpath='{.clusters[].cluster.certificate-authority-data}' \
| base64 -d > ${WORKDIR}/vault.ca



kubectl create secret generic vault-ha-tls \
   -n $VAULT_K8S_NAMESPACE \
   --from-file=vault.key=${WORKDIR}/vault.key \
   --from-file=vault.crt=${WORKDIR}/vault.crt \
   --from-file=vault.ca=${WORKDIR}/vault.ca

Aplicaremos una licencia [Enterprise](https://developer.hashicorp.com/vault/docs/platform/k8s/helm/enterprise) al cluster de Vault

In [ ]:
%%bash
secret=$(cat vault.hclic)
kubectl create secret generic vault-ent-license --from-literal="license=${secret}" -n $VAULT_K8S_NAMESPACE

Creamos el fichero de configuración con el que instalar Vault. Instalaremos lo siguiente:
* Un HA clúster compuesto por 3 nodos con auto_join.
* Cluster con TLS usando certificados firmados por la CA de Kubernetes.
* Tanto los certificados como las licencias se montaran en un volumen
* No aplicaremos (anti-) affinity rules
* Instalaremos Vault agent injector
```yaml
injector:
   enabled: true
   image:
      repository: docker.io/hashicorp/vault-k8s
   agentImage:
      repository: docker.io/hashicorp/vault
````
* Habilitaremos el CSI provider
```yaml
csi:
   enabled: true
   image:
      repository: "docker.io/hashicorp/vault-csi-provider"
```

In [ ]:
%%bash
cat > ${WORKDIR}/overrides.yaml <<EOF
global:
   enabled: true
   tlsDisable: false

csi:
   enabled: true
   image:
      repository: "docker.io/hashicorp/vault-csi-provider"

injector:
   enabled: true
   image:
      repository: docker.io/hashicorp/vault-k8s
   agentImage:
      repository: docker.io/hashicorp/vault

# Supported log levels include: trace, debug, info, warn, error
logLevel: "trace" # Set to trace for initial troubleshooting, info for normal operation

server:
   image:
      repository: docker.io/hashicorp/vault-enterprise
      tag: 1.18.3-ent
   enterpriseLicense:
      secretName: vault-ent-license
   extraEnvironmentVars:
      VAULT_CACERT: /vault/userconfig/vault-ha-tls/vault.ca
      VAULT_TLSCERT: /vault/userconfig/vault-ha-tls/vault.crt
      VAULT_TLSKEY: /vault/userconfig/vault-ha-tls/vault.key
   volumes:
      - name: userconfig-vault-ha-tls
        secret:
         defaultMode: 420
         secretName: vault-ha-tls
   volumeMounts:
      - mountPath: /vault/userconfig/vault-ha-tls
        name: userconfig-vault-ha-tls
        readOnly: true
   standalone:
      enabled: false
   affinity: "" #No affinity rules so I can install 3 vault instances in a single
   ha:
      enabled: true
      replicas: 3
      raft:
         enabled: true
         setNodeId: true
         config: |
            ui = true
            listener "tcp" {
               tls_disable = 0 
               address = "[::]:8200"
               cluster_address = "[::]:8201"
               tls_cert_file = "/vault/userconfig/vault-ha-tls/vault.crt"
               tls_key_file  = "/vault/userconfig/vault-ha-tls/vault.key"
               tls_client_ca_file = "/vault/userconfig/vault-ha-tls/vault.ca"
            }
            storage "raft" {
               path = "/vault/data"
            
               retry_join {
                  auto_join             = "provider=k8s namespace=vault label_selector=\"component=server,app.kubernetes.io/name=vault\""
                  auto_join_scheme      = "https"
                  leader_ca_cert_file   = "/vault/userconfig/vault-ha-tls/vault.ca"
                  leader_tls_servername = "vault-0.vault-internal" #Tiene que matchear una SAN del certificado
               }
            
            }
            telemetry {
               disable_hostname = true
               prometheus_retention_time = "12h"
            }
            disable_mlock = true
            service_registration "kubernetes" {}
      # Vault UI
   ui:
      enabled: true
      serviceType: "LoadBalancer"
      serviceNodePort: null
      externalPort: 8200

   
EOF

Finalmente usando el values.yaml instalamos Vault

In [ ]:
! helm install -n $VAULT_K8S_NAMESPACE $VAULT_HELM_RELEASE_NAME hashicorp/vault -f ${WORKDIR}/overrides.yaml

Verifiquemos la instalación

In [ ]:
! kubectl get events -n vault

In [ ]:
! kubectl -n $VAULT_K8S_NAMESPACE get pods #--watch

Una vez instalado Vault procedemos a su inicialización. Por simplicidad usaremos una sola **unseal key**

In [ ]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE vault-0 -- vault operator init \
    -key-shares=1 \
    -key-threshold=1 \
    -format=json > ${WORKDIR}/cluster-keys.json

Con la unseal key obtenida en el paso previo procederemos a desellar Vault

In [ ]:
%%bash
jq -r ".unseal_keys_b64[]" ${WORKDIR}/cluster-keys.json
VAULT_UNSEAL_KEY=$(jq -r ".unseal_keys_b64[]" ${WORKDIR}/cluster-keys.json)

Nodo 0

In [ ]:
! kubectl exec -n $VAULT_K8S_NAMESPACE vault-0 -- vault operator unseal $(jq -r ".unseal_keys_b64[]" ${WORKDIR}/cluster-keys.json)

Nodo 1

In [ ]:
! kubectl exec -n $VAULT_K8S_NAMESPACE vault-1 -- vault operator unseal $(jq -r ".unseal_keys_b64[]" ${WORKDIR}/cluster-keys.json)

Nodo 2

In [ ]:
! kubectl exec -n $VAULT_K8S_NAMESPACE -ti vault-2 -- vault operator unseal $(jq -r ".unseal_keys_b64[]" ${WORKDIR}/cluster-keys.json)

In [ ]:
! kubectl exec -n $VAULT_K8S_NAMESPACE -ti vault-0 -- vault status

In [ ]:
! kubectl exec -n $VAULT_K8S_NAMESPACE -ti vault-1 -- vault status

In [ ]:
! kubectl exec -n $VAULT_K8S_NAMESPACE -ti vault-2 -- vault status

Por último vamos a hacer un port-foward contra Vault para poder operar y crear un archivo de configuración con la URL y root token de Vault

In [ ]:
%%bash
cat > ${WORKDIR}/config.env <<EOF
VAULT_ADDR=https://127.0.0.1:8200
VAULT_TOKEN=$(cat $WORKDIR/cluster-keys.json | jq -r ".root_token")
VAULT_CACERT=/tmp/vault/vault.ca

EOF

In [ ]:
%%bash
#  kill -9 $(lsof -t -i:8200)
kubectl -n vault port-forward service/vault 8200:8200 